In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df = pd.read_csv('analytics.csv')

In [3]:
df['date_time']=pd.to_datetime(df['last_update']/1000, unit='s')
df['date_time'].head()
df['hours']= df['date_time'].dt.hour

In [4]:
df['day']=df['date_time'].dt.dayofweek

In [13]:
df[df['day']==6].head()

,Unnamed: 0,available_bike_stands,available_bikes,bike_stands,last_update,number,status,latest_weather,clouds,conditions,date,description,sunrise,sunset,temperature,wind,date_time,hours,days,day
24600,24600,12,19,31,1521332012000,1,OPEN,1521329400,75,Rain,1521329400,shower rain,1521354760,1521398072,-0.43,11.3,2018-03-18 00:13:32,0,6,6
24601,24601,16,4,20,1521331710000,2,OPEN,1521329400,75,Rain,1521329400,shower rain,1521354760,1521398072,-0.43,11.3,2018-03-18 00:08:30,0,6,6
24602,24602,19,1,20,1521332015000,3,OPEN,1521329400,75,Rain,1521329400,shower rain,1521354760,1521398072,-0.43,11.3,2018-03-18 00:13:35,0,6,6
24603,24603,20,0,20,1521331637000,4,OPEN,1521329400,75,Rain,1521329400,shower rain,1521354760,1521398072,-0.43,11.3,2018-03-18 00:07:17,0,6,6
24604,24604,31,9,40,1521331414000,5,OPEN,1521329400,75,Rain,1521329400,shower rain,1521354760,1521398072,-0.43,11.3,2018-03-18 00:03:34,0,6,6


In [14]:
#https://stackoverflow.com/questions/19790790/splitting-dataframe-into-multiple-dataframes

#create unique list of names
UniqueNumbers = df.number.unique()

#create a data frame dictionary to store your data frames
DataFrameDictHours = {}

for key in UniqueNumbers:
    for i in range(7):
        for j in range (24):
            DataFrameDictHours[str(key)+'/'+str(i)+'/'+str(j)] = df[:][(df.number == key)&(df.hours==j)&(df.day==i)]

In [15]:
DataFrameDictHours['105/0/19']

,Unnamed: 0,available_bike_stands,available_bikes,bike_stands,last_update,number,status,latest_weather,clouds,conditions,date,description,sunrise,sunset,temperature,wind,date_time,hours,days,day
124074,124074,31,5,36,1522696105000,105,OPEN,1522693800,75,Fog,1522693800,fog,1522648473,1522695799,7.00,3.1,2018-04-02 19:08:25,19,0,0
124075,124075,31,5,36,1522696719000,105,OPEN,1522693800,75,Fog,1522693800,fog,1522648473,1522695799,7.00,3.1,2018-04-02 19:18:39,19,0,0
124591,124591,31,5,36,1522697334000,105,OPEN,1522695600,75,Fog,1522695600,fog,1522648470,1522695801,6.51,2.1,2018-04-02 19:28:54,19,0,0
124592,124592,31,5,36,1522697949000,105,OPEN,1522695600,75,Fog,1522695600,fog,1522648470,1522695801,6.51,2.1,2018-04-02 19:39:09,19,0,0
124593,124593,31,5,36,1522699175000,105,OPEN,1522695600,75,Fog,1522695600,fog,1522648470,1522695801,6.51,2.1,2018-04-02 19:59:35,19,0,0


In [17]:
with open('dict.pkl','wb') as output:
    pickle.dump(DataFrameDictHours, output, pickle.HIGHEST_PROTOCOL)

In [18]:
del DataFrameDictHours

In [7]:
with open('dict.pkl','rb') as input:
    DataFrameDictHours = pickle.load(input)
    print(DataFrameDictHours['105/0/19'])

        Unnamed: 0  available_bike_stands  available_bikes  bike_stands  \
124074      124074                     31                5           36   
124075      124075                     31                5           36   
124591      124591                     31                5           36   
124592      124592                     31                5           36   
124593      124593                     31                5           36   

          last_update  number status  latest_weather  clouds conditions  \
124074  1522696105000     105   OPEN      1522693800      75        Fog   
124075  1522696719000     105   OPEN      1522693800      75        Fog   
124591  1522697334000     105   OPEN      1522695600      75        Fog   
124592  1522697949000     105   OPEN      1522695600      75        Fog   
124593  1522699175000     105   OPEN      1522695600      75        Fog   

              date description     sunrise      sunset  temperature  wind  \
124074  1522693800   

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
105*24*7

17640

In [ ]:
model_dict = {}
for key in DataFrameDictHours.keys():
    if DataFrameDictHours[key][['wind','temperature']].shape[0]==0:
        continue
    if DataFrameDictHours[key]['available_bikes'].shape[0]==0:
        continue
    rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
    model_dict[key]=rfc.fit(DataFrameDictHours[key][['wind','temperature']],DataFrameDictHours[key]['available_bikes'])

/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs 

In [6]:
X=DataFrameDict['105/23'][['wind','temperature']]

In [7]:
y=DataFrameDict['105/23'][['available_bikes']]

In [8]:
#print("Descriptive features:\n", X)
#print("\nTarget feature:\n", y)

#logreg = LinearRegression().fit(X, y)

#predictions = logreg.predict(X)

#print("Accuracy: ", metrics.accuracy_score(y, predictions))
#print("Confusion matrix: \n", metrics.confusion_matrix(y, predictions))
#print("Classification report:\n ", metrics.classification_report(y, predictions))

In [9]:
dtc = DecisionTreeClassifier(max_depth=3, random_state=1)
dtc.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [10]:
pd.DataFrame({'feature': X.columns, 'importance': dtc.feature_importances_})

,feature,importance
0,wind,0.306196
1,temperature,0.693804


In [11]:
# Alternative to print a tree in text format
"""
treeviz.py

A simple tree visualizer for sklearn DecisionTreeClassifiers.

Written by Lutz Hamel, (c) 2017 - Univeristy of Rhode Island
"""
import operator

def tree_print(clf, X):
    """
    Print the tree of a sklearn DecisionTreeClassifier

    Parameters
    ----------
    clf : DecisionTreeClassifier - A tree that has already been fit.
    X : The original training set
    """
    tlevel = _tree_rprint('', clf, X.columns, clf.classes_)
    print('<',end='')
    for i in range(3*tlevel - 2):
        print('-',end='')
    print('>')
    print('Tree Depth: ',tlevel)

def _tree_rprint(kword, clf, features, labels, node_index=0, tlevel_index=0):
    # Note: The DecisionTreeClassifier uses the Tree structure defined in:
    # 		github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/_tree.pyx
    #       it is an array based tree implementation:

    # indent the nodes according to their tree level
    for i in range(tlevel_index):
        print('  |',end='')

	#  TODO: the following should use the TREE_LEAF constant defined in _tree.pyx
	#        instead of -1, not quite sure how to get at it from the tree user level
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        print(kword, end=' ' if kword else '')
        # get the majority label
        count_list = clf.tree_.value[node_index, 0]
        max_index, max_value = max(enumerate(count_list), key=operator.itemgetter(1))
        max_label = labels[max_index]
        print(max_label)
        return tlevel_index
    
    else:
        # compute and print node label
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        print(kword, end=' ' if kword else '')
        print('if {} =< {}: '.format(feature, threshold))
        # recurse down the children
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        ltlevel_index = _tree_rprint('then', clf, features, labels, left_index, tlevel_index+1)
        rtlevel_index = _tree_rprint('else', clf, features, labels, right_index, tlevel_index+1)
        # return the maximum depth of either one of the children
        return max(ltlevel_index,rtlevel_index)

In [12]:
# Use the function tree_print defined above to plot a decision tree in text format
tree_print(dtc, X)

if temperature =< 3.9700000286102295: 
  |then if wind =< 3.3499999046325684: 
  |  |then if temperature =< 2.0: 
  |  |  |then 2
  |  |  |else 7
  |  |else if temperature =< 2.809999942779541: 
  |  |  |then 11
  |  |  |else 2
  |else if temperature =< 5.279999732971191: 
  |  |then 8
  |  |else if wind =< 3.8499999046325684: 
  |  |  |then 4
  |  |  |else 5
<------->
Tree Depth:  3


In [13]:
dtc.predict(X)

array([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8, 11,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  4,  4,  4,  4,  4,  5,  5,  5,
       11, 11, 11,  5,  5,  5])

In [17]:
predictions=dtc.predict(X)

In [18]:
print("Accuracy: ", metrics.accuracy_score(y, predictions))

Accuracy:  0.775


In [20]:
print("MSE: ", metrics.mean_squared_error(y,predictions))

MSE:  7.15


In [21]:
print("MAE: ", metrics.mean_absolute_error(y,predictions))

MAE:  1.15


In [32]:
y=y['available_bikes']

In [38]:
# Evaluate the model using 3-fold cross-validation.
# Uses 2 parts of data for training and the last part for testing. 
# This process is repeated 3 times. More details about cross-validation here: http://www-bcf.usc.edu/~gareth/ISL/
scores = cross_val_score(DecisionTreeClassifier(max_depth=3, random_state=1), X, y, scoring='neg_mean_absolute_error', cv=10)
print(scores)
print(scores.mean())

/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


[-2.75       -1.71428571 -1.71428571 -1.2        -0.         -0.
 -1.         -1.         -1.         -0.        ]
-1.0378571428571428


In [39]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)

In [40]:
rfc.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [46]:
rfc_predictions = rfc.predict(X)

In [43]:
scores = cross_val_score(RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1), X, y, scoring='neg_mean_absolute_error', cv=10)
print(scores)
print(scores.mean())

/home/robbie/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


[-0.875 -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.    -0.   ]
-0.0875


In [44]:
rfc.oob_score_

0.95

In [45]:
105*24

2520